This ipynb is a modified version of [colab-katago-en.ipynb](https://colab.research.google.com/drive/1sKv4QHdB6P-fSNMIhZkaC5gti6qhHDeC?usp=sharing#scrollTo=nPaa2KJt8Kyn) and uses resources which [@kinfkong](https://github.com/kinfkong) san developed.
I really appreciate @kinfkong san.

This document shows how to run KataGo WebSocket Server on Colab.

# Step 1: Run KataGo and WebSocket server on Colab.

Before running, you should prepare these 1 values: `NGROK_TOKEN`

* NGROK_TOKEN - You can register or login in `ngrok` website here: [https://dashboard.ngrok.com/auth/your-authtoken](https://dashboard.ngrok.com/auth/your-authtoken). Then you can find your own ngrok token.

Change NGROK_TOKEN of your own in the below code (the first line). Then click the `Run` button. 

It may take about 2~3 minutes to complete running the code. 

Please click the URL of the last output. Lizzie-like web app will start in new window.

If you have any uncertainty for this step, you can also check this simple tutorial video: 
https://monosnap.com/file/gi2nIGtYXkosLInuDb9j5HQQkwB72w

In [ ]:
NGROK_TOKEN="<Your ngrok authtoken>"

# optional args
# supports: OPENCL, CUDA or AUTO
KATAGO_BACKEND="AUTO"
# supports: 40b, 20b or AUTO
WEIGHT_FILE="AUTO" 

import subprocess
gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
if KATAGO_BACKEND == "AUTO":
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND="CUDA"
  else:
    KATAGO_BACKEND="OPENCL"

if WEIGHT_FILE == "AUTO":
  if gpu_name == "TeslaK80" or gpu_name == "TeslaP4":
    WEIGHT_FILE="20b"
  else:
    WEIGHT_FILE="40b"

!echo "Using Katago Backend : " $KATAGO_BACKEND
!echo "Using Katago Weight : " $WEIGHT_FILE
!echo "GPU : " $gpu_name

# Install useful stuff
!apt-get update 1>/dev/null
#!apt install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
!apt install --yes git libzip4 1>/dev/null
!pip install oss2 1>/dev/null
!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/libzip.so.5.0 -O /usr/lib/x86_64-linux-gnu/libzip.so.5
%cd /content

!rm -rf katago-colab
!git clone https://github.com/kinfkong/katago-colab.git 1>/dev/null

#download the binarires
!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.2/katago-$KATAGO_BACKEND -O katago
!chmod +x /content/katago
!wget -q https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/ngrok -O ngrok
!chmod +x /content/ngrok
!mkdir -p /root/.katago/
!cp -r /content/katago-colab/opencltuning /root/.katago/

#download the weights
if WEIGHT_FILE == "40b" :
  !wget -q https://baduk17.cafe24.com/kataurl.txt
  KATAGO_URL=open("kataurl.txt", mode='r').read()
  !wget -q $KATAGO_URL -O weight.bin.gz
else:
  !wget -q https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b20c256x2-s5303129600-d1228401921.bin.gz -O weight.bin.gz

if KATAGO_BACKEND == "CUDA":
  %cd /usr/lib/x86_64-linux-gnu/
  !wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/libcublas.so.11.3.0.106 -O libcublas.so.11
  !wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/libcublasLt.so.11.3.0.106 -O libcublasLt.so.11
  %cd /content
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/libcudnn8_8.0.5.39-1+cuda11.1_amd64.deb
  !dpkg -i libcudnn8_8.0.5.39-1+cuda11.1_amd64.deb
else:
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv8.txt -O /root/.katago/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv8.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaP4_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaP4_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv8.txt -O /root/.katago/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv8.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv8.txt -O /root/.katago/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv8.txt

!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/gtp_colab.cfg -O /content/gtp_colab.cfg

# start websocket server here
!pip install websockets

script = """\
import argparse
import asyncio
import websockets


def start(cmd, host, port):
    async def cli(websocket, path):
        p = await asyncio.create_subprocess_exec(*cmd, stdin=asyncio.subprocess.PIPE, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
        async def recv():
            async for message in websocket:
                if message.startswith("loadsgf"):
                    tokens = message.split()
                    with open("tmp.sgf", "w") as f:
                        f.write(tokens[1])
                    message = "loadsgf tmp.sgf\\n" if len(tokens) == 2 else f"loadsgf tmp.sgf {tokens[2]}\\n"
                p.stdin.write(message.encode("utf-8"))
        async def send():
            while True:
                line = await p.stdout.readline()
                if line == "":
                    break
                await websocket.send(line.decode("utf-8"))
        async def stderr():
            while True:
                line = await p.stderr.readline()
                line = line.decode("utf-8")
                if line == "":
                    break
                if line.startswith("GTP ready"):
                    await websocket.send(line)

                
        await asyncio.gather(recv(), send(), stderr())
        p.close()

    server = websockets.serve(cli, host, port)
    asyncio.get_event_loop().run_until_complete(server)
    asyncio.get_event_loop().run_forever()

if __name__== "__main__":
    parser = argparse.ArgumentParser(description="invoke websocket server with stdio command.")
    parser.add_argument("command", type=str, help="command (double quote if including arguments)")
    parser.add_argument("--host", type=str, default="127.0.0.1", help="host name (default: localhost)")
    parser.add_argument("--port", type=int, default=3001, help='port number')
    args = vars(parser.parse_args())
    start(args["command"].split(" "), args["host"], args["port"])
"""

with open("websocket_server.py", "w") as f:
  f.write(script)

print("pass1")
#!python3 websocket_server.py "./katago gtp -model weight.bin.gz -config gtp_colab.cfg" &
get_ipython().system_raw('python3 websocket_server.py "./katago gtp -model weight.bin.gz -config gtp_colab.cfg" &')
print("pass2")

# Run ngrok
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok http 3001 &')
!sleep 5

import oss2
import requests
import json
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
websocket = sub("https://", "wss://", r.json()['tunnels'][0]['public_url'])
print(f"https://new3rs.herokuapp.com/a_master_of_go/index.html?gtp={websocket}")



#The following sections are for debugging only, you can ignore.

**Shows the Colab GPU Info**

In [ ]:
!nvidia-smi

Get your ssh login info
*ssh login account is root, the login password is the `USER_PASSWORD` you configured in the previous steps*

In [ ]:
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh 
print(str_ssh)

ssh root@0.tcp.ngrok.io -p 12828
